In [1]:
from pandas.io import gbq
import pandas as pd
import matplotlib.pyplot as plt

bq_project = "thesis-jaeyoon"

In [2]:
gp_query= """
select 
	*
from 
	(
	select
		p.publication_number,
		p.application_number,
		p.country_code,
		p.priority_date,
    p.family_id,
    p.entity_status,
		STRING_AGG(DISTINCT ipcs.code) as ipc_codes,
    STRING_AGG(DISTINCT assignees) as assignees,
    STRING_AGG(DISTINCT uspcs.code) as uspc_codes,
    STRING_AGG(DISTINCT pc.publication_number) as priority_publication_number,
    STRING_AGG(DISTINCT cit.publication_number) as citations,
    STRING_AGG(DISTINCT ah.name) as assignees_harmonized,
    title.text as title_text,
    title.language as title_lang,
    abstract.text as abstract_text,
    abstract.language as abstract_lang
 	FROM
	  `patents-public-data.patents.publications` AS p
  LEFT JOIN UNNEST(p.ipc) AS ipcs
  LEFT JOIN UNNESt(p.uspc) AS uspcs
  LEFT JOIN UNNEST(p.assignee) AS assignees
  LEFT JOIN UNNEST(p.title_localized) AS title
  LEFT JOIN UNNEST(p.abstract_localized) AS abstract
  LEFT JOIN UNNEST(p.priority_claim) AS pc 
  LEFT JOIN UNNEST(p.citation) as cit
  LEFT JOIN UNNEST(p.assignee_harmonized) as ah
	GROUP BY 
    p.publication_number, p.application_number, p.country_code, 
		p.priority_date, p.family_id, p.entity_status, title.text, title.language,
		abstract.text, abstract.language
	)
where 
	ipc_codes LIKE '%A61J1/%' OR ipc_codes LIKE '%A61J3/%' OR
  ipc_codes LIKE '%A61K9/%' OR ipc_codes LIKE '%A61K31/%' OR
  ipc_codes LIKE '%A61K33/%' OR ipc_codes LIKE '%A61K35/%' OR
  ipc_codes LIKE '%A61K36/%' OR ipc_codes LIKE '%A61K41/%' OR
  ipc_codes LIKE '%A61K49/%' OR ipc_codes LIKE '%A61K51/%' OR
  ipc_codes LIKE '%A61K39/%' OR ipc_codes LIKE '%A61K45/%' OR
  ipc_codes LIKE '%A61K47/%' OR ipc_codes LIKE '%A61K48/%' OR
  ipc_codes LIKE '%A61P/%' OR
	uspc_codes LIKE '%424/%' OR uspc_codes LIKE '%514/%' 
"""

In [12]:
gp_query="""

select
	p.publication_number,
	p.application_number,
	p.country_code,
  p.publication_date,
	p.priority_date,
  p.family_id,
  p.entity_status,
	STRING_AGG(DISTINCT ipcs.code) as ipc_codes,
  STRING_AGG(DISTINCT uspcs.code) as uspc_codes,
  STRING_AGG(DISTINCT pc.application_number) as priority_application_number,
  STRING_AGG(DISTINCT cit.publication_number) as citations,
  STRING_AGG(DISTINCT ah.name, "||") as assignees_harmonized,
  STRING_AGG(DISTINCT title.text, "||") as title_text,
  STRING_AGG(DISTINCT abstract.text, "||") as abstract_text
FROM
  `patents-public-data.patents.publications` AS p
LEFT JOIN UNNEST(p.ipc) AS ipcs
LEFT JOIN UNNESt(p.uspc) AS uspcs
LEFT JOIN UNNEST(p.assignee) AS assignees
LEFT JOIN UNNEST(p.title_localized) AS title
LEFT JOIN UNNEST(p.abstract_localized) AS abstract
LEFT JOIN UNNEST(p.priority_claim) AS pc 
LEFT JOIN UNNEST(p.citation) as cit
LEFT JOIN UNNEST(p.assignee_harmonized) as ah
LEFT JOIN UNNEST(p.description_localized) as description
where 
  pc.application_number = p.application_number AND 
  (
  ipcs.code LIKE '%A61J1/%' OR ipcs.code LIKE '%A61J3/%' OR
  ipcs.code LIKE '%A61K9/%' OR ipcs.code LIKE '%A61K31/%' OR
  ipcs.code LIKE '%A61K33/%' OR ipcs.code LIKE '%A61K35/%' OR
  ipcs.code LIKE '%A61K36/%' OR ipcs.code LIKE '%A61K41/%' OR
  ipcs.code LIKE '%A61K49/%' OR ipcs.code LIKE '%A61K51/%' OR
  ipcs.code LIKE '%A61K39/%' OR ipcs.code LIKE '%A61K45/%' OR
  ipcs.code LIKE '%A61K47/%' OR ipcs.code LIKE '%A61K48/%' OR
  ipcs.code LIKE '%A61P/%' OR
  uspcs.code LIKE '%424/%' OR uspcs.code LIKE '%514/%' 
  )
GROUP BY 
  p.publication_number, p.application_number, p.country_code, p.publication_date,
	p.priority_date, p.family_id, p.entity_status
"""

In [13]:
gp_df = gbq.read_gbq(
            query=gp_query,
            project_id=bq_project,
            verbose=True,
            dialect='standard')

/Users/jaeyoonjung/anaconda3/lib/python3.6/site-packages/pandas/io/gbq.py:149: FutureWarning: verbose is deprecated and will be removed in a future version. Set logging level in order to vary verbosity
  credentials=credentials, verbose=verbose, private_key=private_key)


In [19]:
gp_df.application_number.nunique()

1037186

In [20]:
gp_df.publication_number.nunique()

1341859

In [21]:
gp_df.shape

(1341859, 14)

In [22]:
gp_df.to_csv("priority_claims.csv", sep="\t", index=False)

In [32]:
gp_df.iloc[1341843].abstract_text

'L&#39;invention a trait au domaine médical et à l&#39;industrie chimique et pharmaceutique, et concerne en particulier des procédés de production d&#39;un complexe de substances biologiquement actives issu de matières végétales qui présente une activité hypoglycémiante. Le procédé de production d&#39;un complexe de substances biologiquement actives présentant une activité hypoglycémiante consiste à : soumettre de la sarriette à une extraction à l&#39;alcool ; puis concentrer et/ou sécher l&#39;extrait, l&#39;extraction étant mise en oeuvre à l&#39;aide d&#39;alcool à 96% afin d&#39;obtenir un premier extrait et un résidu de sarriette ; concentrer le premier extrait afin d&#39;obtenir un premier extrait solide, ou concentrer et sécher le premier extrait afin d&#39;obtenir un premier extrait sec ; soumettre le résidu de sarriette à une extraction à l&#39;alcool à 70% afin d&#39;obtenir un second extrait, dont au moins une partie est concentrée afin d&#39;obtenir un second extrait solide

In [31]:
gp_df.loc@

1341843    L&#39;invention a trait au domaine médical et ...
Name: abstract_text, dtype: object

In [23]:
gp_df

,publication_number,application_number,country_code,publication_date,priority_date,family_id,entity_status,ipc_codes,uspc_codes,priority_application_number,citations,assignees_harmonized,title_text,abstract_text
0,NL-7407480-A,NL-7407480-A,NL,19751208,19740604,19821485,,A61K31/215,None,NL-7407480-A,None,MERCKLE KG LUDWIG,Para-chlorophenoxy-isobutyrate salt - of dimet...,The novel title cpd. is prepd. by reacting p-c...
1,CA-1098828-A,CA-293483-A,CA,19810407,19771220,4110337,,A61K31/70,None,CA-293483-A,None,TOMIC DOBRIVOJE,Traduction non-disponible||Pharmaceutical prod...,ABSTRACT The invention deals with a new p...
2,FI-891015-A0,FI-891015-A,FI,19890303,19890303,8527991,,"A61K31/13,A61K31/075",None,FI-891015-A,None,ORION YHTYMAE OY,Ett foerbaettrat foerfarande foer framstaellni...,None
3,DK-565288-D0,DK-565288-A,DK,19881011,19881011,8144235,,"A61K9/50,A61K31/23,A61K9/107",None,DK-565288-A,None,NOVO INDUSTRI AS,Fremgangsmaade til fremstilling af triglycerid...,None
4,CA-2408207-A1,CA-2408207-A,CA,20040416,20021016,32097367,,"A61K31/7088,A61K48/00",None,CA-2408207-A,None,PASTEUR INSTITUT,Peptides liant la proteine phosphatase 2a et p...,L&#39;invention a trait à de nouveaux peptides...
5,CA-2538623-C,CA-2538623-A,CA,20100105,20060303,38468968,,A61J1/03,None,CA-2538623-A,None,9155 0020 QUEBEC INC,Feuille de scellement utilisee pour fermer une...,Disclosed is an improved sealing sheet for use...
6,FI-7066-U1,FI-U20050233-U,FI,20060512,20050701,34803146,,A61J1/16,None,FI-U20050233-U,None,TYKE OY,Ställning för injektionsnålsbehållare||Teline ...,None
7,BE-832363-R,BE-159144-A,BE,19751201,19750812,25648779,,A61K51/08,None,BE-159144-A,None,None,Procede de marouage de proteintes a l&#39;aide...,None
8,NL-184872-B,NL-8100739-A,NL,19890703,19810216,19837028,,A61K31/65,None,NL-8100739-A,None,CHINOIN GYOGYSZER ES VEGYESZET,Farmaceutisch preparaat met antimicrobiele wer...,None
9,CA-1044142-A,CA-230772-A,CA,19781212,19750704,4103533,,"A61K31/405,A61K31/135",None,CA-230772-A,None,DAWKINS PETER D||MCARTHUR JOHN N||SMITH MERVYN...,Anti-inflammatory dipeptide||Dipeptide anti-in...,Abstract of the Disclosure The pharmaceutical ...


In [ ]:
gp_df.to_csv("google_patents_claims.csv", index=False, sep='\t')

In [ ]:
gp_df = pd.read_csv("google_patents.csv", sep='\t')

In [ ]:
gp_df.head()

In [ ]:
gp_df.shape

In [ ]:
gp_df = gp_df.drop("priority_publication_number", axis=1)

In [ ]:
gp_df.publication_number.nunique()

In [ ]:
gp_df_clean = gp_df.drop(["title_text", "title_lang", "abstract_text", "abstract_lang"], axis=1)

In [ ]:
gp_df_clean = gp_df_clean.drop_duplicates()

In [ ]:
gp_df_clean.drop_duplicates(["publication_number", "application_number", "country_code", "priority_date", 
       "family_id", "entity_status"],inplace=True)

In [ ]:
gp_df_clean.to_csv("google_patents_clean.csv", index=False, sep='\t')

In [ ]:
gp_df_clean =  pd.read_csv("google_patents_clean.csv", sep='\t')

In [ ]:
gp_df_clean.head()

In [ ]:
gp_df_clean.groupby("country_code")["publication_number"].count()

In [ ]:
gp_df.family_id.nunique()

In [ ]:
sum(gp_df.ipc_codes.isnull())

check FDA Orangebook: any patents missing?

In [ ]:
ob_patent_dir = "orangebook/patent.txt"
ob_patent = pd.read_csv(ob_patent_dir, sep="~")

In [ ]:
ob_patent

In [ ]:
ob_patent["patent_number"] = ob_patent["Patent_No"].apply(lambda x: re.sub("\D", "", x))

In [ ]:
pd.DataFrame(ob_patent.patent_number.unique()).to_csv("orangebook.seed.csv", index=False)

In [ ]:
ob_patent_Nos = list(ob_patent.patent_number.unique())

In [ ]:
len(ob_patent_Nos)

In [ ]:
where_clause = ",".join("'" + ob_patent.patent_number.unique() + "'")

In [ ]:
ob_query = '''
    select
		p.publication_number,
		p.application_number,
		p.country_code,
		p.priority_date,
    p.family_id,
    p.entity_status,
		STRING_AGG(DISTINCT ipcs.code) as ipc_codes,
    STRING_AGG(DISTINCT assignees) as assignees,
    STRING_AGG(DISTINCT uspcs.code) as uspc_codes,
    STRING_AGG(DISTINCT pc.publication_number) as priority_publication_number,
    STRING_AGG(DISTINCT cit.publication_number) as citations,
    STRING_AGG(DISTINCT ah.name) as assignees_harmonized,
    title.text as title_text,
    title.language as title_lang,
    abstract.text as abstract_text,
    abstract.language as abstract_lang
 	FROM
	  `patents-public-data.patents.publications` AS p
  LEFT JOIN UNNEST(p.ipc) AS ipcs
  LEFT JOIN UNNESt(p.uspc) AS uspcs
  LEFT JOIN UNNEST(p.assignee) AS assignees
  LEFT JOIN UNNEST(p.title_localized) AS title
  LEFT JOIN UNNEST(p.abstract_localized) AS abstract
  LEFT JOIN UNNEST(p.priority_claim) AS pc 
  LEFT JOIN UNNEST(p.citation) as cit
  LEFT JOIN UNNEST(p.assignee_harmonized) as ah
    WHERE
        REGEXP_EXTRACT(p.publication_number, r'\w+-(\d+)-\w+') IN
        (
        {}
        )
	GROUP BY 
    p.publication_number, p.application_number, p.country_code, 
		p.priority_date, p.family_id, p.entity_status, title.text, title.language,
		abstract.text, abstract.language
    '''.format(where_clause)

In [ ]:
ob_df = gbq.read_gbq(
            query=ob_query,
            project_id=bq_project,
            verbose=True,
            dialect='standard')

In [ ]:
ob_df.to_csv("ob_patents.csv", index=False, sep='\t')

In [ ]:
ob_df = pd.read_csv("ob_patents.csv", sep='\t')

In [ ]:
ob_df

In [ ]:
ob_df_clean = ob_df.drop(["title_text", "title_lang", "abstract_text", "abstract_lang", "priority_publication_number"], axis=1)

In [ ]:
ob_df_clean = ob_df_clean.drop_duplicates()

In [ ]:
ob_df_clean.drop_duplicates(["publication_number", "application_number", "country_code", "priority_date", 
       "family_id", "entity_status"],inplace=True)

In [ ]:
ob_df_clean.to_csv("ob_patents_clean.csv", index=False, sep='\t')

In [ ]:
all_patents = pd.concat([gp_df_clean, ob_df_clean])

In [ ]:
all_patents = all_patents.reset_index(drop=True)
all_patents.drop_duplicates(["publication_number", "application_number", "country_code", "priority_date", 
       "family_id", "entity_status"],inplace=True)

In [ ]:
all_patents

In [ ]:
#all_patents.to_csv("all_patents.csv", sep='\t', index=False)
all_patents = pd.read_csv("all_patents.csv", sep='\t')

In [ ]:
all_patents.shape

In [ ]:
all_patents.publication_number.nunique()

In [ ]:
3307656 - 3304450

In [ ]:
all_patents["priority_year"]= all_patents["priority_date"].apply(lambda x: int(str(x)[0:4]))
patent_count_by_year = all_patents.groupby(["priority_year", "country_code"])["publication_number"].count()

In [ ]:
patent_count_by_year = patent_count_by_year.reset_index()

In [ ]:
patent_count_by_year = patent_count_by_year.loc[(patent_count_by_year.priority_year >= 1975) & 
                                                (patent_count_by_year.priority_year <= 2012)]

In [ ]:
plt.figure(figsize=(20,20))
countries = patent_count_by_year.country_code.unique()
for c in countries:
    country_patent = patent_count_by_year.loc[patent_count_by_year.country_code==c]
    x=country_patent.priority_year
    y=country_patent.publication_number
    plt.plot(x,y, label=c)
plt.axvline(x=1995, color="red", label="TRIPS")
plt.axvline(x=2002, color="green", label="Doha")
plt.legend()

In [ ]:
all_patent_count_by_year = patent_count_by_year.groupby(["priority_year"])["publication_number"].sum().reset_index()

In [ ]:
plt.figure(figsize=(20,20))

x=all_patent_count_by_year.priority_year
y=all_patent_count_by_year.publication_number
plt.plot(x,y, label="Number of Patnets")
plt.axvline(x=1995, color="red", label="TRIPS")
plt.axvline(x=2002, color="green",label="Doha")
plt.legend()

In [ ]:
patent_count_by_year

## DRUGBANK

In [ ]:
db_df = pd.read_csv("drugbank.csv", sep='\t')

In [ ]:
db_df.head()

In [ ]:
db_df = db_df.loc[~db_df.patents.isnull()].reset_index(drop=True)

In [ ]:
db_patents = db_df.apply(lambda x: pd.Series(x["patents"].split("|")), axis=1).stack().reset_index(level=1, drop=True)

In [ ]:
db_patents.name="patent_id"
db_df = db_df.join(db_patents).reset_index(drop=True)

In [ ]:
db_df["patent_id"] = db_df["patent_id"].apply(int)

In [ ]:
db_df

# add descriptions

In [ ]:
desc_query = """
select 
	publication_number,
  description_text,
  description_lang
from 
	(
	select
		p.publication_number,
    STRING_AGG(DISTINCT ipcs.code) as ipc_codes,
    STRING_AGG(DISTINCT uspcs.code) as uspc_codes,
    description.text as description_text,
    description.language as description_lang
 	FROM
	  `patents-public-data.patents.publications` AS p,
      UNNEST(p.description_localized) AS description
  LEFT JOIN UNNEST(p.ipc) AS ipcs
  LEFT JOIN UNNESt(p.uspc) AS uspcs
	GROUP BY 
    p.publication_number, ipcs.code, uspcs.code, description.text, description.language
	)
where 
	ipc_codes LIKE '%A61J1/%' OR ipc_codes LIKE '%A61J3/%' OR
  ipc_codes LIKE '%A61K9/%' OR ipc_codes LIKE '%A61K31/%' OR
  ipc_codes LIKE '%A61K33/%' OR ipc_codes LIKE '%A61K35/%' OR
  ipc_codes LIKE '%A61K36/%' OR ipc_codes LIKE '%A61K41/%' OR
  ipc_codes LIKE '%A61K49/%' OR ipc_codes LIKE '%A61K51/%' OR
  ipc_codes LIKE '%A61K39/%' OR ipc_codes LIKE '%A61K45/%' OR
  ipc_codes LIKE '%A61K47/%' OR ipc_codes LIKE '%A61K48/%' OR
  ipc_codes LIKE '%A61P/%' OR
	uspc_codes LIKE '%424/%' OR uspc_codes LIKE '%514/%' 
"""

In [ ]:
desc_query = """

	select
		p.publication_number,
        description.text as description_text,
        description.language as description_lang
 	FROM
	  `patents-public-data.patents.publications` AS p,
      UNNEST(p.description_localized) AS description
  LEFT JOIN UNNEST(p.ipc) AS ipcs
  LEFT JOIN UNNESt(p.uspc) AS uspcs
  where
      ipcs.code LIKE '%A61J1/%' OR ipcs.code LIKE '%A61J3/%' OR
      ipcs.code LIKE '%A61K9/%' OR ipcs.code LIKE '%A61K31/%' OR
      ipcs.code LIKE '%A61K33/%' OR ipcs.code LIKE '%A61K35/%' OR
      ipcs.code LIKE '%A61K36/%' OR ipcs.code LIKE '%A61K41/%' OR
      ipcs.code LIKE '%A61K49/%' OR ipcs.code LIKE '%A61K51/%' OR
      ipcs.code LIKE '%A61K39/%' OR ipcs.code LIKE '%A61K45/%' OR
      ipcs.code LIKE '%A61K47/%' OR ipcs.code LIKE '%A61K48/%' OR
      ipcs.code LIKE '%A61P/%' OR
      uspcs.code LIKE '%424/%' OR uspcs.code LIKE '%514/%' 
"""

In [ ]:
desc_df = gbq.read_gbq(
            query=desc_query,
            project_id=bq_project,
            verbose=False,
            dialect='standard')

In [ ]:
desc_df.head()

In [ ]:
desc_df.to_csv("patent_description.csv", index=False, sep='\t')